In [1]:
import numpy as np
import scipy as sc
import pandas as pd
import itertools as it
import subprocess as sp
import logging
import sys
import re
import collections
import json
import pysam
import enum
import bisect
import pgenlib as pg

## options are given as a json object

In [2]:
params = {
    'bam_file_name':
      '/share/PI/mrivas/data/nanopore-wgs-consortium/poretools_fastq.12894489.geq12500.bam',
    'fasta_ext':
      'fa',
    'ref_genome_template':
      '/share/PI/mrivas/data/hg19/chr${CHR}.${EXT}',
    'ref_population_template':
      '/share/PI/mrivas/data/1000genomes/ALL.chr${CHR}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes-pgen.${EXT}'
}

In [3]:
json.dumps(params)

'{"bam_file_name": "/share/PI/mrivas/data/nanopore-wgs-consortium/poretools_fastq.12894489.geq12500.bam", "ref_genome_template": "/share/PI/mrivas/data/hg19/chr${CHR}.${EXT}", "ref_population_template": "/share/PI/mrivas/data/1000genomes/ALL.chr${CHR}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes-pgen.${EXT}", "fasta_ext": "fa"}'

In [4]:
chromosome = 20

## need to look up file name for reference data

In [5]:
def get_ref_file_name(filename_template, chromosome, extension):
    '''Convert filename template to actual file name
    
    filename_template: full path to reference file with meta variables${CHR} ${EXT}
      ${CHR} will be replaced with chromosome
      ${EXT} will be replaced with extension (fa, pgen)
    chromosome:        chromosome
    extension:         extension
    '''    
    return(re.sub(r'\${CHR}(.*)\${EXT}', r'{}\1{}'.format(str(chromosome),
                                                          str(extension)),
                  filename_template))

def get_fasta_file_name(param_obj, chromosome):
    return(get_ref_file_name(param_obj['ref_genome_template'], 
                             chromosome, 
                             param_obj['fasta_ext']))

def get_pgen_file_name(param_obj, chromosome):
    return(get_ref_file_name(param_obj['ref_population_template'], 
                             chromosome, 
                             'pgen'))

def get_bim_file_name(param_obj, chromosome):
    return(get_ref_file_name(param_obj['ref_population_template'], 
                             chromosome, 
                             'bim'))


In [6]:
print get_fasta_file_name(params, 20)
print get_pgen_file_name(params, 20)
print get_bim_file_name(params, 20)

/share/PI/mrivas/data/hg19/chr20.fa
/share/PI/mrivas/data/1000genomes/ALL.chr20.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes-pgen.pgen
/share/PI/mrivas/data/1000genomes/ALL.chr20.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes-pgen.bim


In [7]:
@enum.unique
class Nucleotide(enum.Enum):
    N = -1
    A = 0
    C = 1
    G = 2
    T = 3
    
Mismatch = collections.namedtuple('Mismatch', 'reference_position reference read quality')    

In [8]:
bamfile = pysam.AlignmentFile(filename = params['bam_file_name'], 
                              mode = 'rb')
reference = pysam.FastaFile(get_fasta_file_name(params, 20))

In [9]:
class read:
    '''class to manage mapped read    
    '''
    def __init__(self, aligned_segment, reference):
        # This copy will be removed
        self.aligned_segment = aligned_segment
        
        # find all mismatches
        self.find_mismatches(aligned_segment, reference)
        
        # copy useful info
        self.query_name      = aligned_segment.query_name     
        self.reference_name  = aligned_segment.reference_name
        self.reference_start = aligned_segment.reference_start
        self.reference_end   = aligned_segment.reference_end
        self.length          = aligned_segment.reference_length
        self.mapping_quality = aligned_segment.mapping_quality


    def get_mismatches(self, quality_threshod = 14):
        return([i for i in self.mismatches 
                if i.quality >= quality_threshod and
                   i.reference is not Nucleotide['N']])

    def n_mismatch(self, quality_threshod = 14):
        return(len(self.get_mismatches(quality_threshod)))
    
    def n_match(self, quality_threshod = 14):
        return(self.length - self.n_mismatch(quality_threshod))
    
        
    def find_mismatches(self, aligned_segment, reference):
        '''find all mismatches
        [args]
        pysam.AlignedSegment aligned_segment: mapped fragment
        pysam.FastxFile      reference:       reference sequence
        '''
        aligned_pairs = np.array(aligned_segment.get_aligned_pairs(matches_only=True, with_seq=True))
        
        # fetch corresponding reference sequence
        reference_str = reference.fetch(reference = aligned_segment.reference_name,
                                        start     = aligned_segment.reference_start, 
                                        end       = aligned_segment.reference_end).upper()
        
        # obtain nucleotide letters on both read and reference
        read_letters = np.array([aligned_segment.query_sequence[int(read_position)].upper()
                                 for read_position in aligned_pairs[:, 0]])
        ref_letters  = np.array([reference_str[int(ref_position) - 
                                               aligned_segment.reference_start] 
                                 for ref_position in aligned_pairs[:, 1]])

        # enumerate all the mismatches by comparing nucleotide letters
        self.mismatches = [Mismatch(reference_position = \
                                      int(aligned_pairs[mismatch_pos_on_pairs][1]),
                                    reference = \
                                      Nucleotide[ref_letters[mismatch_pos_on_pairs]],
                                    read      = \
                                      Nucleotide[read_letters[mismatch_pos_on_pairs]],
                                    quality   = \
                                      aligned_segment.query_qualities[int(aligned_pairs[mismatch_pos_on_pairs][0])])
                           for mismatch_pos_on_pairs 
                           in np.where(read_letters != ref_letters)[0]]

    def __str__(self):
        return('\t'.join(['{}:{}-{}'.format(self.reference_name, 
                                            self.reference_start,
                                            self.reference_end),
                          str(self.n_match()),
                          str(self.n_mismatch()),
                          str(self.mapping_quality),
                          self.query_name]))

In [10]:
#aligned_segment = bamfile.next()
aligned_segments = [bamfile.next() for i in range(10)]

In [11]:
r = read(aligned_segments[0], reference)

In [12]:
print(r)

chr20:59980-65177	5190	7	60	33ecb953-edd6-445b-87da-85c4551d8c8c_Basecall_Alignment_template


In [13]:
r.get_mismatches()

[Mismatch(reference_position=60992, reference=<Nucleotide.G: 2>, read=<Nucleotide.T: 3>, quality=14),
 Mismatch(reference_position=61171, reference=<Nucleotide.A: 0>, read=<Nucleotide.T: 3>, quality=14),
 Mismatch(reference_position=61220, reference=<Nucleotide.G: 2>, read=<Nucleotide.A: 0>, quality=17),
 Mismatch(reference_position=63344, reference=<Nucleotide.T: 3>, read=<Nucleotide.A: 0>, quality=15),
 Mismatch(reference_position=63377, reference=<Nucleotide.C: 1>, read=<Nucleotide.A: 0>, quality=16),
 Mismatch(reference_position=63378, reference=<Nucleotide.C: 1>, read=<Nucleotide.A: 0>, quality=16),
 Mismatch(reference_position=63798, reference=<Nucleotide.C: 1>, read=<Nucleotide.T: 3>, quality=15)]

## bim file

In [14]:
class bim_file():
    def __init__(self, params):
        self.params  = params
        self.raw_tables  = {}
        self.id = {}
        self.morgan = {}
        self.bp = {}
        self.pri = {}
        self.sec = {}
        
    def load_raw_table(self, chromosome):
        self.raw_tables[str(chromosome)] = \
            pd.read_csv(get_bim_file_name(chromosome=chromosome,
                                          param_obj = self.params),
                        sep = '\t', 
                        names = ['chr', 'id', 'morgan', 'bp', 'pri', 'sec'])            
        self.id[    str(chromosome)] = np.array(self.raw_tables[str(chromosome)].ix[:, 1])            
        self.morgan[str(chromosome)] = np.array(self.raw_tables[str(chromosome)].ix[:, 2])            
        self.bp[    str(chromosome)] = np.array([int(x) for x in self.raw_tables[str(chromosome)].ix[:, 3]])
        self.pri[   str(chromosome)] = np.array(self.raw_tables[str(chromosome)].ix[:, 4])
        self.sec[   str(chromosome)] = np.array(self.raw_tables[str(chromosome)].ix[:, 5])            
        
    def is_loaded(self, chromosome):
        return(str(chromosome) in self.raw_tables)
        
    def get_raw_table(self, chromosome):
        if(not self.is_loaded(chromosome)):
            self.load_raw_table(chromosome)
        return(self.raw_tables[str(chromosome)])  
    
    def get_id(self, chromosome):
        if(not self.is_loaded(chromosome)):
            self.load_raw_table(chromosome)
        return(self.id[str(chromosome)])  

    def get_morgan(self, chromosome):
        if(not self.is_loaded(chromosome)):
            self.load_raw_table(chromosome)
        return(self.morgan[str(chromosome)])  
    
    def get_bp(self, chromosome):
        if(not self.is_loaded(chromosome)):
            self.load_raw_table(chromosome)
        return(self.bp[str(chromosome)])  

    def get_pri(self, chromosome):
        if(not self.is_loaded(chromosome)):
            self.load_raw_table(chromosome)
        return(self.pri[str(chromosome)])  

    def get_sec(self, chromosome):
        if(not self.is_loaded(chromosome)):
            self.load_raw_table(chromosome)
        return(self.sec[str(chromosome)])  
    
    def find_index(self, chromosome, position):
        bp = self.get_bp(chromosome)
        return(min(len(bp) - 1,
                   max(0,
                       bisect.bisect_left(bp, position))))
    
    def find_index_interval(self, chromosome, pos_l, pos_r):
        '''Find semi-open interval of indicies of SNPs that 
           overlaps with a mapped fragment spanning [pos_l, pos_r)
        '''
        index_l = self.find_index(chromosome, pos_l)
        index_r = self.find_index(chromosome, pos_r)
        return((index_l, index_r))

In [15]:
bim = bim_file(params)

In [16]:
bim.get_id(20)

array(['rs527639301', 'rs538242240', 'rs149529999', ..., 'rs563604166',
       'rs577023641', 'rs546194182'], dtype=object)

In [17]:
bim.get_bp(20)

array([   60343,    60419,    60479, ..., 62965290, 62965305, 62965354])

In [18]:
print bim.find_index_interval(20, 60342, 60419)
print bim.find_index_interval(20, 60343, 60420)
print bim.find_index_interval(20, 60344, 60421)

(0, 1)
(0, 2)
(1, 2)


In [19]:
[x for x in range(1, 2)]

[1]

## pgen file for population reference

In [28]:
class Haplotype:
    def __init__(self, chromosome, index_l, index_r, hap_ary):
        self.chromosome = chromosome
        self.index_l = index_l
        self.index_r = index_r
        self.hap_ary = hap_ary
        
    def get_bitstr(self):
        return(''.join([str(x) if x in [0, 1] else '?' 
                        for x in self.hap_ary]))
    
    def __hash__(self):
        return(hash((self.chromosome,
                     self.index_l,
                     self.index_r,
                     self.get_bitstr())))
    
    def __eq__(self, other):
        return(self.__hash__() == other.__hash__())

    def __ne__(self, other):        
        return(not(self == other))

In [32]:
class population_reference():
    def __init__(self, params):
        self.params  = params
        self.pgens   = {}
        
    def load_pgen(self, chromosome):
        self.pgens[str(chromosome)] =\
            pg.PgenReader(get_pgen_file_name(chromosome=chromosome,
                                             param_obj = self.params))   

    def is_loaded(self, chromosome):
        return(str(chromosome) in self.pgens)
        
    def get_pgen(self, chromosome):
        if(not self.is_loaded(chromosome)):
            self.load_pgen(chromosome)
        return(self.pgens[str(chromosome)])
    
    def read_haplotype(self, chromosome, index_l, index_r):
        pgen = self.get_pgen(chromosome)
        sample_ct = pgen.get_raw_sample_ct()
        alleles_list = np.zeros((index_r - index_l, 2 * sample_ct),
                                dtype = np.int32)
        pgen.read_alleles_range(index_l, index_r, alleles_list)
        return([Haplotype(chromosome, index_l, index_r, alleles_list[:, i]) 
                for i in range(alleles_list.shape[1])])

In [33]:
pgen = population_reference(params)

In [34]:
# the first read spans
r.reference_name, r.reference_start, r.reference_end

('chr20', 59980, 65177)

In [35]:
# indices are 
bim.find_index_interval(r.reference_name[3:], r.reference_start, r.reference_end)

(0, 134)

In [36]:
# haplotypes are
haps = pgen.read_haplotype(r.reference_name[3:], 0, 134)

In [38]:
len(set(haps))

170

# TODO
- manage haplotype block
  - partition bim file
- inference